In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from os import path
import os
import glob

np.seterr(all='raise')


def metis_partition_file_converter(inp, out):
    import sys


    partitions = {}

    def add(el, idx):
        if idx not in partitions:
            partitions[idx] = [el]
        else:
            partitions[idx].append(el)


    with open(inp, "r+") as f:
        el = 0
        for line in f:
            idx = int(line.strip())
            add(el, idx)
            el += 1

    with open(out, "w+") as f:
        idxs = list(partitions.keys())
        idxs.sort()
        assert(idxs[0] <= idxs[-1])
        for ix in idxs:
            f.write(" ".join([str(e) for e in partitions[ix]]) + "\n")


import numpy as np

def graph_and_cut_to_numpy(gf, cf):

    graph = {}
    with open(gf, "r") as f:
        first_row = f.readline().strip("\n").split(" ")
        while [] in first_row:
            first_row.remove([])
        n, e = [int(e) for e in first_row]
        for i, row in enumerate(f.readlines()):
            graph[i + 1] = []
            for v in row.strip("\n").split(" "):
                if v == "":
                    continue
                assert(int(v))
                graph[i + 1].append(int(v))
                try:
                    graph[v].append(int(i + 1))
                except KeyError:
                    graph[v] = [int(i + 1)]

    clusters = []
    #TOFIX: clusters are 0 index
    with open(cf, "r") as f:
        n_counter = 0
        e_counter = 0
        for i, row in enumerate(f.readlines()):
            clusters.append([int(e) + 1 for e in row.strip("\n").split(" ") if e != ""])
            #print(clusters[-1])
            n_counter += 1
        e_counter += len(clusters[-1])


    return graph, clusters


def test(graph_f, cut_f):
    graph, cuts = graph_and_cut_to_numpy(graph_f, cut_f)
    #graph, cuts = graph_and_cut_to_numpy("graphs/whitaker3.graph", "old_results/whitaker3.graph.part.94.chaco")
    subgs = []

    for clidx, cut in enumerate(cuts):

        print("Start working on cluster;", clidx)
        if len(cut) == 1:
            print("Cut len is 1, continue")
            continue

        v_set = set()
        for el in cut:
            v_set.add(int(el))

        #print(v_set)
        n = len(v_set)

        subg = {k:[] for k in cut}
        #print("first element in cut", cut[0])
        for k in graph:
            for v in graph[k]:
                if v in v_set and k in v_set:
                    subg[k].append(v)


        rekey_dict    = {v:i for i, v in enumerate(sorted(list(subg.keys())))}
        rekey_reverse = {i:v for i, v in enumerate(sorted(list(subg.keys())))}

        success = True
        #new_subg = [[0 for _ in range(n)] for _ in range(n)]
        new_subg = sparse.dok_matrix((n, n))

        for k in subg: 

            if len(subg[k]) == 0:
                print("WARNING - disconnected cluster;", clidx, "number of nodes in subgraph is;", len(cut))
                success = False
                break

            for v in subg[k]:
                #new_subg[rekey_dict[k]][rekey_dict[v]] = 1
                #new_subg[rekey_dict[v]][rekey_dict[k]] = 1
                new_subg[rekey_dict[k], rekey_dict[v]] = 1
                new_subg[rekey_dict[v], rekey_dict[k]] = 1

        if not success:
            continue


        steps = test_convergence(new_subg, 0.01)

        print("Cluster", clidx, "of size: ", n, "took n steps to converge;", steps)

        import math
        print("steps/ log2 nodes;", steps/math.log2(n))

        #Scipy and numpy can't calculate eigenvalues quick
        
        n = len(graph)
    if True: #n <= 5000
        trivial_subg = sparse.dok_matrix((n, n))
        for i, k in enumerate(graph):
            for j, _ in enumerate(graph[k]):
                #trivial_subg[i][j] = 1
                trivial_subg[i, j] = 1

        steps = test_convergence(trivial_subg, 0.01)
        print("Whole graph took n steps to converge:", steps)
        import math
        print("steps/ log2 nodes", steps/math.log2(n))
        


import glob


def run_decomp(graph_files):
    for graph in graph_files:
        print("decomp on;", graph)
        print(graph)

        #run decomposition on all graphs with time limit
        out_path = graph + ".out"
        #os.system
        #subprocess.check_call('time -p timeout 15m ./a.out  -S --G_phi=0.01 --H_phi=0.4 --vol=1 --h_ratio=0. -f $graph > "$out_path"', shell=True)
        try:
            subprocess.check_call("time -p timeout 15m ./a.out  -S --G_phi=0.01 --H_phi=0.4 --vol=1 --h_ratio=0. -f " +  graph + " > " + out_path, shell=True)
        except:
            #Too long time?
            continue

        #TODO return code
        if not glob.glob(graph + "cut.txt"):
            print("decomp on", graph, "did not finish in time")
            continue

        #how many clusters, k, did we get?
        with open(out_path) as f:
            lns = f.readlines()
            cluster_line = next(l for l in lns if "n clusters" in l)
            cluster_line.strip("\n")
            n_clusters = int(cluster_line.split(";")[1])

        if n_clusters <= 1:
            print("No cut found, continue")
            continue

        print("n clusters found", n_clusters)
        #run metis on k
        metis_stdio_path = graph + ".out.metis"


        try:
            subprocess.check_call("/usr/bin/gpmetis -ufactor=1000 " +  graph + " " + str(n_clusters) + " -contig >  " + metis_stdio_path, shell=True)
            metis_file = graph + ".part." + str(n_clusters)
            decomp_file = graph + "cut.txt"
            
            metis_partition_file_converter(metis_file, metis_file)
            print("decomp metis")
            test(graph, metis_file)
        except Exception as e:
            print("METIS failed", e)

        rw_graph      = graph + ".row_whole"
        rw_file_ours  = graph + ".rw_ours"
        rw_file_metis = graph + ".rw_metis"

        #TOFIX This should be saving to file!
        print("decomp ours")
        test(graph, decomp_file)
        print("decomp metis")
        test(graph, metis_file)        

        bname = os.path.basename(graph).split(".")[0]
        os.system('mkdir -p results/"$bname"')
        for f in glob.glob("".join(graph.split(".")[:-1]) + "*"):
            if f != graph:
                os.system('mv $f results/"$bname"/')


"""
import glob


g1s = list(glob.glob("synthetic/*"))
run_decomp(g1s)
g2s = list(glob.glob("graphs/*"))
run_decomp(g2s)
"""

# In[ ]:






In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import scipy.sparse as sparse
import torch
import scipy
import time

def test_convergence(matrix, threshold):

    n = matrix.shape[0]
    e = matrix.sum()

    print("n;", n)
    print("e;", e)

    #Number of in-links
    #Axii are inverted for some reason
    colsums = [i.item() for i in scipy.array((matrix.sum(axis=0))).T]

    uniform = np.zeros(shape=(n,))

    #Stationary distribution is uniform normalized by in-links
    for i in range(n):
        uniform[i] = colsums[i]/e
    
    assert(0.99 <= uniform.sum() <= 1.01)
    assert(-0.01 < sum(colsums) - e < 0.01)

    #TODO seed the random start
    walk = np.array([1.] + [0 for _ in range(n - 1)], np.float64)
    walk = torch.FloatTensor(walk)

    #Every node has positive in and out link
    sums = matrix.sum(axis=1)
    assert((sums != 0).any())
    sums = matrix.sum(axis=0)
    assert((sums != 0).any())

    rowsums = matrix.sum(axis=1)
    for i in range(n):
        matrix[i, i] = rowsums[i].item()
        cnt = rowsums[i] * 2
        matrix[i] /= cnt
        #assert(0.999 <= matrix[i].sum() <= 1.001)

    matrix = matrix.tocoo()   
    values = matrix.data
    indices = np.vstack((matrix.row, matrix.col))

    #Number of edges and self-loops
    assert(len(matrix.row) == len(matrix.col) == e + n)

    matrix = torch.sparse.FloatTensor(torch.LongTensor(indices), torch.FloatTensor(values), torch.Size(matrix.shape))

    all_ones = torch.FloatTensor([1. for n in range(n)])

    assert(torch.allclose(all_ones, torch.sparse.sum(matrix, dim=1).to_dense()))

    uniform = torch.from_numpy(uniform)
    dist = torch.norm(walk - uniform, 1)

    steps = 0

    if torch.cuda.is_available():        
        matrix   = matrix.to("cuda")
        walk     = walk.to("cuda")
        uniform  = uniform.to("cuda")
        all_ones = all_ones.to("cuda")

    print("Start walk")


    step_lim = n**2
    print_max = 15
    timeout_minutes = 10

    time_start = time.time()
    
    while dist > threshold and steps < step_lim: 
        #Why is this transposed?
        walk = torch.sparse.mm(matrix.transpose(1, 0), walk.unsqueeze(-1)).squeeze(-1)

        assert(0.99  < torch.sum(walk)    < 1.01)
        assert(0.99  < torch.sum(uniform) < 1.01)
        assert(-0.02 < torch.sum(dist)    < 2.02)
        assert(0.99  < torch.sum(uniform) < 1.01)
        assert(-0.01 < torch.sum(walk - uniform) < 0.01)

        steps += 1
        assert(walk.shape == uniform.shape)
        dist = torch.norm(walk - uniform, 1)

        if steps % (step_lim // print_max) == 0:
            print("Dist at", steps, " = ", dist)
            #print("Walk - uniform", walk - uniform)

        if (time.time() - time_start) > timeout_minutes:
            print("No convergence! ")
            return -1

    print("dist;", dist)

    return steps


"""
g1s = list(glob.glob("synthetic/*"))
g1s.sort()
run_decomp(g1s)
g2s = list(glob.glob("graphs/*"))
g2s.sort()
run_decomp(g2s)
"""
gls = list(glob.glob("results/*/*graphcut*"))
gls.sort()
for gcut in gls:
    bname = os.path.basename(gcut)
    graph = os.path.join("graphs/", bname[:-7]) #x.graphcut.txt -> x.graph
    test(graph, gcut)

